<a href="https://colab.research.google.com/github/hoquangthaiholy/ply/blob/master/Demonstration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installation

In [1]:
!git clone https://github.com/hoquangthaiholy/ply.git
%cd ply
!make install
%cd ..

# Fix Interactive Mode
__file__ = '/content'

Cloning into 'ply'...
remote: Enumerating objects: 2021, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 2021 (delta 66), reused 81 (delta 30), pack-reused 1878
Receiving objects: 100% (2021/2021), 1.09 MiB | 14.35 MiB/s, done.
Resolving deltas: 100% (1243/1243), done.
/content/ply
python3 -m pip install .
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/ply
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing 

#Lex and Yacc Example

In [2]:
# -----------------------------------------------------------------------------
# example.py
#
# Example of using PLY To parse the following simple grammar.
#
#   expression : term PLUS term
#              | term MINUS term
#              | term
#
#   term       : factor TIMES factor
#              | factor DIVIDE factor
#              | factor
#
#   factor     : NUMBER
#              | NAME
#              | PLUS factor
#              | MINUS factor
#              | LPAREN expression RPAREN
#
# -----------------------------------------------------------------------------

from ply import lex
from ply import yacc

# --- Tokenizer

# All tokens must be named in advance.
tokens = ( 'PLUS', 'MINUS', 'TIMES', 'DIVIDE', 'LPAREN', 'RPAREN',
           'NAME', 'NUMBER' )

# Ignored characters
t_ignore = ' \t'

# Token matching rules are written as regexs
t_PLUS = r'\+'
t_MINUS = r'-'
t_TIMES = r'\*'
t_DIVIDE = r'/'
t_LPAREN = r'\('
t_RPAREN = r'\)'
t_NAME = r'[a-zA-Z_][a-zA-Z0-9_]*'

# A function can be used if there is an associated action.
# Write the matching regex in the docstring.
def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

# Ignored token with an action associated with it
def t_ignore_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

# Error handler for illegal characters
def t_error(t):
    print(f'Illegal character {t.value[0]!r}')
    t.lexer.skip(1)

# Build the lexer object
lexer = lex.lex()
    
# --- Parser

# Write functions for each grammar rule which is
# specified in the docstring.
def p_expression(p):
    '''
    expression : term PLUS term
               | term MINUS term
    '''
    # p is a sequence that represents rule contents.
    #
    # expression : term PLUS term
    #   p[0]     : p[1] p[2] p[3]
    # 
    p[0] = ('binop', p[2], p[1], p[3])

def p_expression_term(p):
    '''
    expression : term
    '''
    p[0] = p[1]

def p_term(p):
    '''
    term : factor TIMES factor
         | factor DIVIDE factor
    '''
    p[0] = ('binop', p[2], p[1], p[3])

def p_term_factor(p):
    '''
    term : factor
    '''
    p[0] = p[1]

def p_factor_number(p):
    '''
    factor : NUMBER
    '''
    p[0] = ('number', p[1])

def p_factor_name(p):
    '''
    factor : NAME
    '''
    p[0] = ('name', p[1])

def p_factor_unary(p):
    '''
    factor : PLUS factor
           | MINUS factor
    '''
    p[0] = ('unary', p[1], p[2])

def p_factor_grouped(p):
    '''
    factor : LPAREN expression RPAREN
    '''
    p[0] = ('grouped', p[2])

def p_error(p):
    print(f'Syntax error at {p.value!r}')

# Build the parser
parser = yacc.yacc()

# Parse an expression
ast = parser.parse('2 * 3 + 4 * (5 - x)')
print(ast)

('binop', '+', ('binop', '*', ('number', 2), ('number', 3)), ('binop', '*', ('number', 4), ('grouped', ('binop', '-', ('number', 5), ('name', 'x')))))


#Other examples

In [3]:
%cd ply/example/

/content/ply/example


In [9]:
!cat BASIC/hello.bas

5  REM HELLO WORLD PROGAM
10 PRINT "HELLO WORLD"
99 END



In [5]:
!python BASIC/basic.py BASIC/hello.bas

HELLO WORLD


In [10]:
!cat BASIC/gcd.bas

10 PRINT "A","B","C","GCD"
20 READ A,B,C
30 LET X = A
40 LET Y = B
50 GOSUB 200
60 LET X = G
70 LET Y = C
80 GOSUB 200
90 PRINT A, B, C, G
100 GOTO 20
110 DATA 60, 90, 120
120 DATA 38456, 64872, 98765
130 DATA 32, 384, 72
200 LET Q = INT(X/Y)
210 LET R = X - Q*Y
220 IF R = 0 THEN 300
230 LET X = Y
240 LET Y = R
250 GOTO 200
300 LET G = Y
310 RETURN
999 END


In [6]:
!python BASIC/basic.py BASIC/gcd.bas

A              B              C              GCD
60             90             120            30
38456          64872          98765          1
32             384            72             8


In [11]:
!cat BASIC/powers.bas

5 PRINT "THIS PROGRAM COMPUTES AND PRINTS THE NTH POWERS"
6 PRINT "OF THE NUMBERS LESS THAN OR EQUAL TO N FOR VARIOUS"
7 PRINT "N FROM 1 THROUGH 7"
8 PRINT
10 FOR N = 1 TO 7
15 PRINT "N = "N
20 FOR I = 1 TO N
30 PRINT I^N,
40 NEXT I
50 PRINT
60 PRINT
70 NEXT N
80 END


In [7]:
!python BASIC/basic.py BASIC/powers.bas

THIS PROGRAM COMPUTES AND PRINTS THE NTH POWERS
OF THE NUMBERS LESS THAN OR EQUAL TO N FOR VARIOUS
N FROM 1 THROUGH 7

N =  1
1              

N =  2
1              4              

N =  3
1              8              27             

N =  4
1              16             81             256            

N =  5
1              32             243            1024           3125           

N =  6
1              64             729            4096           15625          46656          

N =  7
1              128            2187           16384          78125          279936         823543         



In [12]:
!cat BASIC/func.bas

10 DEF FDX(X) = 2*X
20 FOR I = 0 TO 100
30 PRINT FDX(I)
40 NEXT I
50 END


In [13]:
!python BASIC/basic.py BASIC/func.bas

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200


In [14]:
!cat BASIC/rand.bas

10 FOR I = 1 TO 20
20 PRINT INT(10*RND(0))
30 NEXT I
40 END


In [15]:
!python BASIC/basic.py BASIC/rand.bas

5
8
6
7
9
7
9
6
0
7
6
7
1
0
0
6
1
7
3
7
